## Cosine Similarity to do movie recommendation

In [60]:
import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

import joblib

from flask import Flask, request, render_template

import json
import bs4
import urllib.request
import requests


In [72]:
# Loading the model and tfidf vectorizer ffrom the disk

vector= joblib.load('NLP_vector.pkl')
model=joblib.load('NLP_model.pkl')

def create_similarity():
    data=pd.read_csv('main_data.csv')
    
    #creating a count matrix
    cv=CountVectorizer()
    count_matrix=cv.fit_transform(data['comb'])
    
    # creating similarity score matrix
    similarity=cosine_similarity(count_matrix)
    return data,similarity


In [74]:
# a=input('write a review: ')

# vector=vector.transform([a]).toarray()  #[a] is converting to list
# my_pred=model.predict(vector)
# # print(my_pred)

# if my_pred==1:
#     print('Review is positive')
# else:
#     print('Review is negative')

In [44]:
data,similarity=create_similarity()

In [45]:
data

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title,comb
0,James Cameron,CCH Pounder,Joel David Moore,Wes Studi,Action Adventure Fantasy Sci-Fi,avatar,CCH Pounder Joel David Moore Wes Studi James C...
1,Gore Verbinski,Johnny Depp,Orlando Bloom,Jack Davenport,Action Adventure Fantasy,pirates of the caribbean: at world's end,Johnny Depp Orlando Bloom Jack Davenport Gore ...
2,Sam Mendes,Christoph Waltz,Rory Kinnear,Stephanie Sigman,Action Adventure Thriller,spectre,Christoph Waltz Rory Kinnear Stephanie Sigman ...
3,Christopher Nolan,Tom Hardy,Christian Bale,Joseph Gordon-Levitt,Action Thriller,the dark knight rises,Tom Hardy Christian Bale Joseph Gordon-Levitt ...
4,Doug Walker,Doug Walker,Rob Walker,UNKNOWN,Documentary,star wars: episode vii - the force awakens ...,Doug Walker Rob Walker UNKNOWN Doug Walker Doc...
...,...,...,...,...,...,...,...
6137,Robert Rodriguez,Priyanka Chopra Jonas,Pedro Pascal,YaYa Gosselin,Action Fantasy Family Comedy,we can be heroes,Priyanka Chopra Jonas Pedro Pascal YaYa Gossel...
6138,Paul Greengrass,Tom Hanks,Helena Zengel,UNKNOWN,Action Adventure Drama Western,news of the world,Tom Hanks Helena Zengel UNKNOWN Paul Greengras...
6139,Regina King,Kingsley Ben-Adir,Eli Goree,Aldis Hodge,Drama,one night in miami...,Kingsley Ben-Adir Eli Goree Aldis Hodge Regina...
6140,Emerald Fennell,Carey Mulligan,Bo Burnham,Alison Brie,Thriller Crime Drama,promising young woman,Carey Mulligan Bo Burnham Alison Brie Emerald ...


In [46]:
#functions to get movie title from movie index and vice versa

def get_titlefromindex(movieindex):
    return data[data.index==movieindex]['movie_title'].values[0]

def get_indexfromtitle(title):
    return data[data.movie_title==title].index.values[0]


In [47]:
# function to take a movie and return the recommendations

def recommendation(movie_user_likes):
    movie_user_likes=movie_user_likes.lower()

    data,similarity=create_similarity()
        
    if movie_user_likes not in data['movie_title'].unique():
        return ('Sorry!, The movie you requested is not in our database. Please check the spelling or try with some other movies' )
    else:
        movie_index=get_indexfromtitle(movie_user_likes)
        similar_movies=list(enumerate(similarity[movie_index]))
        sorted_similar_movies=sorted(similar_movies,key=lambda x:x[1],reverse=True)[1:] 
        # [1:] to remove first movie from the sorted list as it's the searched movie itself
        
        #list of recommended movies
        recommended_movies = []
        for mov in (sorted_similar_movies[0:10]):
            recommended_movies.append(get_titlefromindex(mov[0]))
        return recommended_movies
    
    

In [48]:
movie_user_likes='venom'
type(movie_user_likes)

str

In [49]:
recommended_movies=recommendation(movie_user_likes)


In [50]:
for i in recommended_movies:
    print(i.title())

Spider-Man: Far From Home
Gangster Squad
Rocknrolla
The New Mutants
Jason Bourne
This Means War
What Happens In Vegas
The Dark Knight Rises
30 Minutes Or Less
Zombieland: Double Tap


In [ ]:
# web scraping to get user reviews from IMDB site
    
# source= urllib.request.urlopen('https://www.imdb.com/title/{}/reviews?ref_=tt_ov_rt'.format(imdb_id)).read